# [Create Agent with Function Call](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview#create-agent-with-function-call) using FunctionTool + ToolOutput
You can enhance your Agents by defining **callback functions as function tools**. These can be provided to `create_agent` via either the `toolset` parameter or the **combination** of `tools and tool_resources`. Here are the distinctions:

1. **toolset**: When using the toolset parameter, you provide not only the function definitions and descriptions but also their implementations. The SDK will execute these functions within `create_and_run_process` or `streaming`. These functions will be invoked based on their definitions.
2. **tools and tool_resources**: When using the tools and tool_resources parameters, only the function definitions and descriptions are provided to `create_agent`, without the implementations. The `Run` or event handler of stream will raise a `requires_action` status based on the function definitions. **Your code must handle this status and call the appropriate functions**.<br/>

As a reference point, let's see how we were used to managed this with the notebook [Assistant_APIs_02 (Function Calling).ipynb](https://github.com/maurominella/openai/blob/main/assistantsapi/Assistant_APIs_02%20(Function%20Calling).ipynb)

# Constants

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

project_endpoint = os.environ["AZURE_AIF_PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://aif01ais1lrx7.services.ai.azure.com/api/projects/aif01project01lrx7>
azure-ai-projects library installed version: 1.0.0b12
azure-ai-agents library installed version: 1.0.1


# Create AI Foundry Agents Client

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client

# Define some custom functions

In [3]:
from datetime import date

def save_file(joke:str) -> str:
    """
    Saves a text file using the textual content passed in the joke variable.
    Args:
        joke (str): The text content to be saved into the file. 
        
    Returns:
        Optional[str]: Returns 'success: file saved' on success, otherwise returns an error message.
    """
    file_name = "joke.txt"
    try:
        with open(file_name, "w") as f:
            f.write(joke)
        return "success: file saved"
    except Exception as e:
        return f"error: {str(e)}"


def get_flights(date_1:date, date_2:date) -> str:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)


def my_cat_born_date() -> str:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)


def send_email(to:str, subject:str, body:str) -> str:
    """ Sends an email """
    import requests, json
    url = 'https://prod-18.swedencentral.logic.azure.com:443/workflows/4f7a19b041e04a9e8ea47303e1af503c/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=TX-eDahoU_QIEOjw9qOXjRyPNqA9s4IVkd0osbsyzzI'  

    headers = {
        'Content-Type': 'application/json'
    }

    data = {
        "to": to,
        "subject": subject,
        "body": body
    }
    response = {"response": str(requests.post(url, headers=headers, data=json.dumps(data)))}
    return json.dumps(response)

# Consolidate the custom functions into a single set

In [4]:
from typing import Any, Callable, Set

user_functions: Set[Callable[..., Any]] = {
    save_file, 
    get_flights, 
    my_cat_born_date,
    send_email
}

user_functions

{<function __main__.get_flights(date_1: datetime.date, date_2: datetime.date) -> str>,
 <function __main__.my_cat_born_date() -> str>,
 <function __main__.save_file(joke: str) -> str>,
 <function __main__.send_email(to: str, subject: str, body: str) -> str>}

# Just for testing: use the `user_function` set to call its functions

In [5]:
for function in user_functions:    
    if function.__name__ == "my_cat_born_date":
        result = function()
    elif function.__name__ == "save_file": 
        result = function("This is the content")
    elif function.__name__ == "get_flights": 
        result = function("2015-01-01", "2021-04-04")
    elif function.__name__ == "send_email": 
        result = function("mauromi@microsoft.com", "email from Python Agent", "body")

    print(f"{function.__name__}() returned {result}")

send_email() returned {"response": "<Response [200]>"}
my_cat_born_date() returned {"cat_born_date": "2015-07-01"}
save_file() returned success: file saved
get_flights() returned {"flights": 2285}


# Initialize `FunctionTool`

In [14]:
from azure.ai.agents.models import FunctionTool, ToolSet

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
agents_client.enable_auto_function_calls(tools=toolset)

print(f"functions.definitions: {functions.definitions}")
print(f"\nfunctions.resources: {functions.resources}")

print(f"functions.definitions: {toolset.definitions}")
print(f"\nfunctions.resources: {toolset.resources}")

functions.definitions: [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}}}, {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'ob

In [15]:
# definitions pretty print
def format_tool_definitions(definitions):
    import json
    def custom_serializer(obj):
        return obj.__dict__ if hasattr(obj, '__dict__') else str(obj)

    function_dict = {item['function']['name']: item['function'] for item in functions.definitions}

    return json.dumps(function_dict, indent=2, default=custom_serializer)

# print(format_tool_definitions (functions.definitions))

# Create AI Foundry Agent

In [16]:
functions.definitions

[{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}}},
 {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type': 'object', 'properties': {'joke': {'type': 'string', 'description': 'No description'}}, 'required': ['joke']}}},
 {'type': 'function', 'function': {'name': 'get_flights', 'description': 'Returns the number of flights in a date interval  ', 'parameters': {'type': 'object', 'properties':

In [17]:
# Create agent with AI search tool and process assistant run
agent = agents_client.create_agent(
    model=deployment_name,
    name="functiontool-agent",
    instructions="You are a helpful assistant",
    tools=functions.definitions
)
print(f"Created agent, ID: {agent.id}")
agent.items

Created agent, ID: asst_XRoI4D6tDbGZNLjJa0Y7x4MT


<bound method _MyMutableMapping.items of {'id': 'asst_XRoI4D6tDbGZNLjJa0Y7x4MT', 'object': 'assistant', 'created_at': 1751365996, 'name': 'functiontool-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters': {'type': 'object', 'properties': {}, 'required': []}, 'strict': False}}, {'type': 'function', 'function': {'name': 'save_file', 'description': 'Saves a text file using the textual content passed in the joke variable.', 'parameters': {'type

# Create the thread and attach a new message to it

In [18]:
# Create a thread
thread = agents_client.threads.create()
print(f"Created thread: {thread}\n")


# Add a user message to the thread
from azure.ai.agents.models import MessageRole

message = agents_client.messages.create(
    thread_id=thread.id, 
    role=MessageRole.USER, 
    content="Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.",
)
print(f"Created message: {message}")


# let's see the messages associated with the thread

list(agents_client.messages.list(thread_id=thread.id))

Created thread: {'id': 'thread_XKVh4lMA0cP5ba1v6Ow0IkFz', 'object': 'thread', 'created_at': 1751365997, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_xIYZkNmOFZz5cMZzoAvPXcC5', 'object': 'thread.message', 'created_at': 1751365997, 'assistant_id': None, 'thread_id': 'thread_XKVh4lMA0cP5ba1v6Ow0IkFz', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


[{'id': 'msg_xIYZkNmOFZz5cMZzoAvPXcC5', 'object': 'thread.message', 'created_at': 1751365997, 'assistant_id': None, 'thread_id': 'thread_XKVh4lMA0cP5ba1v6Ow0IkFz', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.', 'annotations': []}}], 'attachments': [], 'metadata': {}}]

# Run the agent synchronously and check the status looping until the run stops
**Note**: if you get "incomplete" due to content filtering, please try running the same cell again

In [19]:
%%time

import time

# Run the agent
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# run.status or run["status"]? it's up to you, but be aware to use it 
# use the run as json object, or as azure.ai.projects.models._models.ThreadRun
while run["status"] in ['queued', 'in_progress', 'cancelling']: # here I use the run as a json object
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

if run["status"] == "failed": 
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")
elif run["status"] == "incomplete":
    print(f'"Incomplete" status returned due to the following reason: <{run["incomplete_details"]["reason"]}>. You may try to run this cell again.\n')
else:
    print(f"Run details:\n{run}\n\nRun finished with status: <{run.status}>. Please proceed by running the next cell.\n")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_ShCa3p1ZjIMcXUYH8MBMQ8e7', 'object': 'thread.run', 'created_at': 1751366001, 'assistant_id': 'asst_XRoI4D6tDbGZNLjJa0Y7x4MT', 'thread_id': 'thread_XKVh4lMA0cP5ba1v6Ow0IkFz', 'status': 'completed', 'started_at': 1751366011, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1751366011, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'send_email', 'description': 'Sends an email ', 'parameters': {'type': 'object', 'properties': {'to': {'type': 'string', 'description': 'No description'}, 'subject': {'type': 'string', 'description': 'No description'}, 'body': {'type': 'string', 'description': 'No description'}}, 'required': ['to', 'subject', 'body']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'my_cat_born_date', 'description': "Returns my cat's born date ", 'parameters'

# KEEP RUNNING this cells until run.status == 'COMPLETED'

In [20]:
# just for checking: analyze the current status

from azure.ai.agents.models import ToolOutput, RequiredFunctionToolCall

run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
print(f"Initial run status: {run.status}")
if not run.required_action is None:
    print(f"\nRequired action(s): {run.required_action}")
    print(f"\nWe need to run {len(run.required_action.submit_tool_outputs.tool_calls)} tool call(s): {run.required_action.submit_tool_outputs.tool_calls}")
    
    i = 0
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
        if isinstance(tool_call, RequiredFunctionToolCall):
            try:
                i += 1
                output = functions.execute(tool_call)
                print(f"\n{i} - Executing tool_call {tool_call.function.name} ({tool_call.id}) >>> output: {output}")
                tool_outputs.append(
                    ToolOutput(
                        tool_call_id=tool_call.id,
                        output=output
                    )
                )
            except Exception as e:
                print(f"Error executing tool_call {tool_call.id}: {e}")
    
    print(f"Tool outputs: {tool_outputs}")

    if tool_outputs:    
        run = agents_client.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
    )
    
    while run["status"] in ["queued", "in_progress"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"\nRun status: {run.status}")
    
    if run["status"] == "requires_action":
        print("\nPlease run this cell again until all required actions are executed.")
    elif run["status"] == "completed":
        print("\nGreat job! All required actions are completed. Please move on to the next cell.")
else:
    print("No action is required")

Initial run status: RunStatus.COMPLETED
No action is required


# Fetch messages from the thread after the agent run execution

In [21]:
from azure.ai.agents.models import (MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, \
    MessageTextFilePathAnnotation, MessageTextUrlCitationAnnotation)

image_files = []
annotations = []
citations = []
citation_annotations = []

if run.status == 'completed':
    messages = agents_client.messages.list(thread_id=thread.id)
    messages_list = list(agents_client.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
                    elif type(a) is MessageTextUrlCitationAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        citation_annotations.append(a)
                    else:
                        print(f"Unknown type {type(a)}")
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)
            else:
                print(f"Unknown type {type(a)}")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) --> Text: Please write into a file the nr of flights between my cat born date and Easter 2021. Send the answer to mauromi@microsoft.com, also.

===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) --> Text: The file has been saved and the email has been sent successfully.


In [22]:
# Get the last message from the sender
last_msg = list(agents_client.messages.list(thread_id=thread.id))[0]

if last_msg:
    print(last_msg.content[0].text.value)

The file has been saved and the email has been sent successfully.


# Print citation_annotations from the messages

In [23]:
print (f"Nr. of citation_annotations: {len(annotations)}\n")

i=0
for ca in citation_annotations:
    i += 1
    print(f"{i} - citation_annotations {ca.text}: {ca.url_citation} (from {ca.start_index} to {ca.end_index})")

Nr. of citation_annotations: 0



# Print annotations from the messages

In [24]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    project_client.agents.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of file path annotations: 0



# Fetch citations from the messages

# Retrieve and download eventual images

In [25]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 0



In [26]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    project_client.agents.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of image contents: 0



# Run Steps

In [27]:
run_steps = list(agents_client.run_steps.list(run_id=run.id, thread_id=thread.id))

print(f'Nr of run step(s): {len(run_steps)}\n')
i=0
for rs in run_steps:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 4

Run step 1: {'id': 'step_4IdUE1YVF85eiEHrAbKTQW07', 'object': 'thread.run.step', 'created_at': 1751366011, 'run_id': 'run_ShCa3p1ZjIMcXUYH8MBMQ8e7', 'assistant_id': 'asst_XRoI4D6tDbGZNLjJa0Y7x4MT', 'thread_id': 'thread_XKVh4lMA0cP5ba1v6Ow0IkFz', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1751366011, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_q85ygO9cKHBStFYYG7QSEiVc'}}, 'usage': {'prompt_tokens': 621, 'completion_tokens': 15, 'total_tokens': 636, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_ihkWj2nNFbbBRcx585udBvau', 'object': 'thread.run.step', 'created_at': 1751366007, 'run_id': 'run_ShCa3p1ZjIMcXUYH8MBMQ8e7', 'assistant_id': 'asst_XRoI4D6tDbGZNLjJa0Y7x4MT', 'thread_id': 'thread_XKVh4lMA0cP5ba1v6Ow0IkFz', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 17

# Collect all resources for this project

In [28]:
all_agents = list_all_agents(client=agents_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=agents_client)
print(all_threads["summary"])

all_files = list_all_files(client=agents_client)
print(all_files["summary"])

all_runs = list_all_runs(client=agents_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(agents_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(agents_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=agents_client)
print(all_vectorstores["summary"])

3 agents
3 threads
0 files
3 runs in 3 threads
0 vector stores


# Teardown for all resources

In [29]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    agents_client.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=agents_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [30]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(client=agents_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [31]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    agents_client.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(client=agents_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_XKVh4lMA0cP5ba1v6Ow0IkFz> has been deleted
2 - Thread <thread_eIlNsc00bYAvL8AwQcGt5E9i> has been deleted
3 - Thread <thread_AAWpSP7I84gHthiEomEbB1i5> has been deleted
Threads deleted: 3



In [32]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    agents_client.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=agents_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_XRoI4D6tDbGZNLjJa0Y7x4MT> has been deleted
2 - Agent <asst_duIdCkbw6S5FbDmFTtfSkZZD> has been deleted
3 - Agent <asst_DSjzbDyJALqYG7mGT0FlxGql> has been deleted
Agents deleted: 3



# HIC SUNT LEONES